# Projet numérique : câble sous marin

## __Réponse 1__

D'après la loi des grands nombre, si $(X_n)_{n \in \mathbb{N}}$ est une suite de variables aléatoires indépendantes, de même loi et intégrables alors $\left (M_n =\frac{1}{n}\sum_{i=0}^n X_i \right ) \xrightarrow[n \rightarrow \infty]{} E(X_1)$ presque sûrement et en moyenne.

## __Réponse 2__

On a $Z = (Z(x_0), ... ,Z(x_n))$. On suppose que l'on connaisse $n-k+1 \in [|1,n-1|]$ composantes de $Z$. On peut supposer sans perdre de généralité que l'on connait $Y = (Z(x_{k}, ... ,Z(x_n))$. Posons $X = (Z(x_0), ... ,Z(x_{k-1}))$. Alors on a 
$$f_{X|Y=y}(x) = \frac{1}{(2\pi)^{k/2}\sqrt{det(CS_X)}}e^{-\frac{1}{2}(x-\psi(y))^t CS_X^{-1}(x - \psi(y))}$$

où $CS_X = Cov(X,X) - Cov(X,Y)Cov(Y,Y)^{-1}Cov(Y,X)$ et $\psi(y) = E(X) + Cov(X,Y)Cov(Z,Z)^{-1}(Y-E(Y))$

## __Réponse 3__

Soit ${Y} = (Y_1,...,Y_p)$ un vecteur de composantes gaussiennes centrées réduites indépendantes. Soit $m\in \mathbb{R}^p$ et $R \in \mathbb{M}_p(\mathbb{R})$. Posons $\textbf{Z} = m +R\textbf{Y}$.

On a $\Phi_Z(u) = E(e^{i \langle u|\textbf{Z} \rangle}) = E(e^{i \langle u |m + R\textbf{Y} \rangle}) = e^{i \langle u | m \rangle}E(e^{i \langle u | R\textbf{Y} \rangle}) = e^{i \langle u | m \rangle}E(e^{i \langle R^t u | \textbf{Y} \rangle}) = e^{i \langle u | m \rangle}\Phi_Y(R^tu)$.

Or $Y$ est un vecteur avec $E(Y) = (0,...,0)$ et $Var(Y) = I_p$. Donc $\Phi_Y(u) = e^{-\frac{u^2}{2}\|u\|}$.

Ainsi $\Phi_Z(u) = e^{i \langle u | m \rangle - \frac{u^2}{2}\langle u | RR^t u \rangle}$.

Donc comme la fonction caractéristique détermine la loi, $Z$ est un vecteur gaussien de moyenne $m$ et de matrice de covariance $RR^t$..